In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

For this preliminary analysis, I will only use wave 2,4,6. \
Each wave's data was collected on 2020-06-17 - 2020-06-23, 2020-11-29 - 2020-12-16, 2021-05-12 - 2021-05-25.

In [3]:
df_w1 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave1_unweighted.csv')
df_w2 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave2_unweighted.csv')
df_w3 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave3_unweighted.csv')
df_w4 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave4_unweighted.csv')
df_w5 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave5_unweighted.csv')
df_w6 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave6_unweighted.csv')

/Users/chrissoria/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
if "resp_employ" in df_w2.columns:
    df_w2["resp_employ"] = df_w2["resp_employ"].astype(str)

if "resp_employ" in df_w4.columns:
    df_w4["resp_employ"] = df_w4["resp_employ"].astype(str)

if "resp_employ" in df_w6.columns:
    df_w6["resp_employ"] = df_w6["resp_employ"].astype(str)
    
df_w2['resp_occupation'] = df_w2['resp_occupation'].astype(str)
df_w4['resp_occupation'] = df_w4['resp_occupation'].astype(str)

df_combined = pd.concat([df_w2, df_w4, df_w6], ignore_index=True)

We're missing a "conservative" but we have two moderates?

In [5]:
political_view_mapping = {
    'Extremely conservative': 1,
    'Slightly conservative': 2,
    'Moderate': 3,
    'Middle of the road': 3,
    'Slightly liberal': 4,
    'Liberal': 5,
    'Extremely liberal': 6
}

# Creating a new variable 'political_view_numeric' by mapping the 'political_view' column using the defined mapping
df_combined['political_view_numeric'] = df_combined['political_view'].replace(political_view_mapping)

df_combined['political_view'].value_counts()

Moderate                  2845
Liberal                   1716
Slightly conservative     1485
Extremely liberal         1381
Extremely conservative    1273
Middle of the road        1204
Slightly liberal           939
Name: political_view, dtype: int64

In [6]:
#education variable
def categorize_education(row):
    if row['resp_educ'] == "Less than high school degree":
        return "Less than high school"
    elif row['resp_educ'] in ["High school graduate (high school diploma or equivalent including GED)", "Some college but no degree", "Associate degree in college (2-year)"]:
        return "High school graduate"
    elif row['resp_educ'] in ["Bachelor's degree in college (4-year)", "Master's degree", "Doctoral degree", "Professional degree (JD, MD)"]:
        return "College graduate and above"
    else:
        return "Unknown"

df_combined['educ_group'] = df_combined.apply(categorize_education, axis=1)

df_combined['educ_group'].value_counts(dropna=False)

race_columns = [col for col in df_combined.columns if col.startswith('resp_race_')]
df_combined[race_columns] = df_combined[race_columns].fillna(0)

#race variable
def categorize_race(row):
    if row['resp_race_1'] == "White":
        return "White"
    elif row['resp_race_2'] == "Black or African American":
        return "Black"
    elif row['resp_race_4'] == "Asian":
        return "Asian"
    else:
        return "Other / Mixed"

df_combined['r_race'] = df_combined.apply(categorize_race, axis=1)

print(df_combined['r_race'].value_counts(dropna=False))

#employment variable
employment_columns = [col for col in df_combined.columns if col.startswith('resp_employ_')]
df_combined[employment_columns] = df_combined[employment_columns].fillna(0)

# Create the 'r_working' var based whether reported
df_combined['r_working'] = np.where(df_combined['industry'] == "I don't work", "Not Working", "Working")

print(df_combined['r_working'].value_counts(dropna=False))

White            8409
Black            1176
Asian             756
Other / Mixed     502
Name: r_race, dtype: int64
Working        7229
Not Working    3614
Name: r_working, dtype: int64


dependent variable for concern

In [7]:
df_combined['binary_concern'] = df_combined['covid19_concern'].apply(
    lambda x: 1 if x in ["Somewhat concerned", "Very concerned"] else (0 if pd.notna(x) else None)
)

df_combined['binary_concern_strong'] = df_combined['covid19_concern'].apply(
    lambda x: 1 if x == "Very concerned" else (0 if pd.notna(x) else None)
)

df_combined['contact_reduction'] = df_combined['covid19_f2fchange'].apply(
    lambda x: 1 if x == "I have greatly reduced face-to-face interaction with others" else (0 if pd.notna(x) else None)
)

reading in the alter file

In [8]:
df_w2_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave2_unweighted.csv")
df_w4_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave4_unweighted.csv")
df_w6_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave6_unweighted.csv")

df_alters_combined = pd.concat([df_w2_nonhhalters, df_w4_nonhhalters, df_w6_nonhhalters], ignore_index=True)

what is num_cc_nonhh? \
Should I use this or how many reported contacts?

In [9]:
# Create binary indicators for mask, gloves, and other protective equipment usage
df_alters_combined['Mask_Used'] = (
    ((df_alters_combined['protection_1'] == "Wear a face mask") |
    (df_alters_combined['protection_2'] == "Wear a face mask") |
    (df_alters_combined['protection_3'] == "Wear a face mask") |
    (df_alters_combined['protection_4'] == "Wear a face mask")).astype(int)
)

df_alters_combined['Gloves_Used'] = (
    ((df_alters_combined['protection_1'] == "Wear gloves") |
    (df_alters_combined['protection_2'] == "Wear gloves") |
    (df_alters_combined['protection_3'] == "Wear gloves") |
    (df_alters_combined['protection_4'] == "Wear gloves")).astype(int)
)

df_alters_combined['Other_Protective_Equipment_Used'] = (
    ((df_alters_combined['protection_1'] == "Wear other protective equipment") |
    (df_alters_combined['protection_2'] == "Wear other protective equipment") |
    (df_alters_combined['protection_3'] == "Wear other protective equipment") |
    (df_alters_combined['protection_4'] == "Wear other protective equipment")).astype(int)
)

# Aggregate these indicators at the 'rid' level
df_aggregated = df_alters_combined.groupby('rid').agg(
    Total_Masks_Used=('Mask_Used', 'sum'),
    Total_Gloves_Used=('Gloves_Used', 'sum'),
    Total_Other_Protective_Equipment_Used=('Other_Protective_Equipment_Used', 'sum'),
    Contacts=('rid', 'size')
).reset_index()

# Normalize the counts by the number of contacts
df_aggregated['Norm_Masks_Used'] = df_aggregated['Total_Masks_Used'] / df_aggregated['Contacts']
df_aggregated['Norm_Gloves_Used'] = df_aggregated['Total_Gloves_Used'] / df_aggregated['Contacts']
df_aggregated['Norm_Other_Protective_Equipment_Used'] = df_aggregated['Total_Other_Protective_Equipment_Used'] / df_aggregated['Contacts']

# Calculate non-weighted and weighted safety indices
df_aggregated['Non_Weighted_Safety_Index'] = (
    df_aggregated['Norm_Masks_Used'] + 
    df_aggregated['Norm_Gloves_Used'] + 
    df_aggregated['Norm_Other_Protective_Equipment_Used']
) / 3

df_aggregated['Weighted_Safety_Index'] = (
    df_aggregated['Norm_Masks_Used'] + 
    df_aggregated['Norm_Gloves_Used'] + 
    df_aggregated['Norm_Other_Protective_Equipment_Used']
) / (3 * df_aggregated['Contacts'])

In [10]:
df_combined_contacts = pd.merge(df_combined, df_aggregated, on = "rid", how = "left")

In [11]:
df_combined_contacts.to_csv('data/BICS_ego_alters_merged_w2_w4_w6.csv', index = False)
df_combined_contacts.to_csv('data/Partisanship and Health Behavior/Data/BICS_ego_alters_merged_w2_w4_w6.csv', index = False)

In [12]:
col_types = {
    'ZIP': str,
    'CD':str,
    'COUNTY_FIPS':str,
    'STATE_FIPS':str,
    'CONGRESSPERSON_PARTY': str
}

ZIP_FEATURES = pd.read_csv('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/data/ZIP_Features.csv',
                          dtype=col_types)

In [13]:
col_types = {
    'resp_zip': str,
    'political_party': str
}

all_waves = pd.read_csv('data/BICS_ego_alters_merged_w2_w4_w6.csv', 
                        low_memory=False, dtype=col_types)

all_waves = all_waves.rename(columns={
    'resp_zip': 'ZIP'
})

columns_to_keep = ['ResponseId','StartDate','resp_yob','resp_sex','resp_hispanic','r_race','resp_nativity','ZIP',
                  'resp_hhsize','r_working','resp_occupation','lefthome_num','num_cc_nonhh',
                  'lefthome_where_1','lefthome_where_2','lefthome_where_3','lefthome_where_4','lefthome_where_10',
                  'lefthome_where_8','lefthome_where_9','lefthome_where_5','lefthome_where_11','lefthome_where_6',
                  'lefthome_where_7','inet_freq','socmedia_use','covid19_familiar','covid19_concern',
                  'covid19_f2fchange','covid19_reduceOK','policy_sip','age','hhi','political_party','political_view',
                  'industry','health_insurance','interview_date','wave','agecat','city','covid19_vax','covid19_whynot_vax',
                  'Non_Weighted_Safety_Index','Weighted_Safety_Index','Norm_Masks_Used','educ_group','contact_reduction',
                  'binary_concern','binary_concern_strong','resp_educ','resp_sex','Contacts','Non_Weighted_Safety_Index',
                  'Weighted_Safety_Index']
all_waves = all_waves[columns_to_keep]

In [14]:
BICS_ZIP_FEATURES = pd.merge(all_waves, ZIP_FEATURES, on = "ZIP", how = "left")
BICS_ZIP_FEATURES

,ResponseId,StartDate,resp_yob,resp_sex,resp_hispanic,r_race,resp_nativity,ZIP,resp_hhsize,r_working,...,POPESTIMATE2021,DEATHS2021,COV_County_CMR_2020_06_20,COV_County_CMR_2020_12_09,COV_County_CMR_2021_05_19,COUNTY_RUCC_2013,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020
0,R_10ARlAAmJ8eGNJb,2020-06-18 21:18:16,1996,Female,No,White,The United States,80011,1,Working,...,654900.0,5228.0,0.517469,0.725067,1.122309,1.0,65.89,20200617.0,65.89,65.89
1,R_10ARlAAmJ8eGNJb,2020-06-18 21:18:16,1996,Female,No,White,The United States,80011,1,Working,...,654900.0,5228.0,0.517469,0.725067,1.122309,1.0,58.59,20201129.0,58.59,58.59
2,R_10ARlAAmJ8eGNJb,2020-06-18 21:18:16,1996,Female,No,White,The United States,80011,1,Working,...,654900.0,5228.0,0.517469,0.725067,1.122309,1.0,65.89,20200617.0,65.89,65.89
3,R_10ARlAAmJ8eGNJb,2020-06-18 21:18:16,1996,Female,No,White,The United States,80011,1,Working,...,654900.0,5228.0,0.517469,0.725067,1.122309,1.0,58.59,20201129.0,58.59,58.59
4,R_yrmrreDsETFkB3j,2020-06-18 21:18:03,1999,Female,No,Asian,The United States,28690,1,Working,...,87611.0,1237.0,0.194184,0.776735,1.746356,2.0,62.24,20200617.0,62.24,62.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30630,R_3G0RIc556HS65Y5,2021-05-26 14:25:26,1985,Male,No,White,The United States,30339,2,Working,...,766802.0,5819.0,0.298874,0.664310,1.387581,1.0,54.95,20201129.0,54.95,54.95
30631,R_2hL2c0J6jveA1wd,2021-05-26 17:48:09,1982,Male,No,Asian,Another country,1545,3,Working,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30632,R_1q21nl5RsYQJxJu,2021-05-26 17:59:11,1967,Male,No,Black,The United States,2368,4,Not Working,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30633,R_3LXR7ok4yurdCMN,2021-05-26 21:03:26,2000,Male,Yes,White,The United States,85032,3,Working,...,4496588.0,41160.0,0.142396,0.949228,2.218126,1.0,52.60,20200617.0,52.60,52.60


In [15]:
BICS_ZIP_FEATURES['political_party_to_CD'] = BICS_ZIP_FEATURES.apply(
    lambda row: str(row['political_party']) + " in " + str(row['CONGRESSPERSON_PARTY']) + " led CD", axis=1
)
BICS_ZIP_FEATURES = BICS_ZIP_FEATURES[~BICS_ZIP_FEATURES['political_party_to_CD'].str.contains('Prefer not to answer')]

# Print the updated DataFrame
print(BICS_ZIP_FEATURES[['political_party', 'CONGRESSPERSON_PARTY', 'political_party_to_CD']])

      political_party CONGRESSPERSON_PARTY             political_party_to_CD
4         Independent           Republican  Independent in Republican led CD
5         Independent           Republican  Independent in Republican led CD
6         Independent           Republican  Independent in Republican led CD
7         Independent           Republican  Independent in Republican led CD
8         Independent           Democratic  Independent in Democratic led CD
...               ...                  ...                               ...
30629     Independent           Republican  Independent in Republican led CD
30630     Independent           Republican  Independent in Republican led CD
30632        Democrat                  NaN            Democrat in nan led CD
30633      Republican           Republican   Republican in Republican led CD
30634      Republican           Republican   Republican in Republican led CD

[28987 rows x 3 columns]


In [16]:
BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'] = BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'].replace("Democratic", "Democrat")
BICS_ZIP_FEATURES['In_Opposing_Party_CD'] = (BICS_ZIP_FEATURES['political_party'] != BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY']).astype(int)
BICS_ZIP_FEATURES['Independent'] = (BICS_ZIP_FEATURES['political_party'] == "Independent").astype(int)
BICS_ZIP_FEATURES['Vaccinated'] = BICS_ZIP_FEATURES['covid19_vax'].apply(
    lambda x: 1 if x == "Yes, I have received at least one dose of a vaccine" else
              (0 if x == "No, I have not" else np.nan)
)

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_33384/212919122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'] = BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'].replace("Democratic", "Democrat")
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_33384/212919122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BICS_ZIP_FEATURES['In_Opposing_Party_CD'] = (BICS_ZIP_FEATURES['political_party'] != BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY']).astype(int

Does it make sense to drop Independents? \
They're kind of in their own cateogory

In [17]:
print(BICS_ZIP_FEATURES['In_Opposing_Party_CD'].value_counts())
print(BICS_ZIP_FEATURES['political_party'].value_counts())
print(BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'].value_counts())

1    15779
0    13208
Name: In_Opposing_Party_CD, dtype: int64
Democrat       13709
Republican      8285
Independent     6993
Name: political_party, dtype: int64
Democrat       18718
Republican      9036
Libertarian       26
Independent       24
Name: CONGRESSPERSON_PARTY, dtype: int64


In [18]:
BICS_ZIP_FEATURES.to_csv('data/BICS_ZIP_Features.csv',index=False)

In [19]:
BICS_ZIP_FEATURES.to_csv('data/Partisanship and Health Behavior/Data/BICS_ZIP_Features.csv',index=False)